# Stable baselines interface
I made a quick interface to stable baselines in case someone needed a jumping off point. Haven't really tested it too much. Should be fine with both discrete and continuous variants

In [1]:
import numpy as np
import os

import tensorflow as tf
from tensorboardX import SummaryWriter

from stable_baselines.common.env_checker import check_env
from stable_baselines import PPO2

from envs.ADCS_gym_disc import AttitudeControlEnv

C:\Users\jakee\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
############## Hyperparameters ##############
log_interval = 10  # print avg reward in the interval
max_episodes = 10000  # max training episodes
max_timesteps = 500  # max timesteps in one episode

update_timestep = 4000  # update policy every n timesteps
K_epochs = 80  # update policy for K epochs
eps_clip = 0.2  # clip parameter for PPO
gamma = 0.99  # discount factor

num_tests = 10

lr = 0.0001  # parameters for Adam optimizer

random_seed = None
#############################################
logpath = './log/20201130_journal/'
runname = '20201202_ppo_sb_testing'

#############################################
os.makedirs(logpath, exist_ok=True)

writer = SummaryWriter(logpath + runname)

# creating environment
env = AttitudeControlEnv(torque_scale=0.5, steps=max_timesteps)

In [3]:
policy_kwargs = dict(act_fun=tf.nn.relu, net_arch=[dict(pi=[400, 300],
                                                          vf=[400, 300])])

In [4]:
model = PPO2("MlpPolicy", env, policy_kwargs=policy_kwargs, verbose=0)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [5]:
def check_reward(num_tests, env, model):
    avg_rew_list = []
    min_ang_list = []

    for testx in range(num_tests):
        rew_list = []
        obs_list = []
        done = False
        state = env.reset()
        while not done:
            # Running policy_old:
            action, __states = model.predict(state)
            state, reward, done, _ = env.step(action)
            rew_list.append(reward)
            obs_list.append(state)
            
        q4 = [i[0] for i in obs_list]
        q1 = [i[1] for i in obs_list]
        q2 = [i[2] for i in obs_list]
        q3 = [i[3] for i in obs_list]

        rewsum = np.sum(rew_list)
        avg_rew_list.append(rewsum)
        
        curr_minang = 2*np.arccos(np.max(q4))*(180/np.pi)
        min_ang_list.append(curr_minang)

    curr_avg_reward = np.mean(avg_rew_list)
    curr_min_ang = np.mean(min_ang_list)

    return curr_avg_reward, curr_min_ang

In [6]:
# training loop
for i_episode in range(1, max_episodes + 1):
    
    # run 1 episode
    model.learn(max_timesteps)      

    # logging
    if i_episode % log_interval == 0:
        
        curr_avg_reward, curr_min_ang = check_reward(num_tests, env, model)

        writer.add_scalar("curr avg reward", curr_avg_reward, i_episode)
        writer.add_scalar("curr avg ang", curr_min_ang, i_episode)

        print(f'Episode {i_episode} \t Avg reward: {curr_avg_reward} \t avg ang: {curr_min_ang}')

Episode 10 	 Avg reward: -254.18446941977646 	 avg ang: 80.88902320418617
Episode 20 	 Avg reward: -178.5126992957728 	 avg ang: 54.62765249215111


KeyboardInterrupt: 